##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hola, muchos mundos

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/quantum/tutorials/hello_many_worlds"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

En este tutorial se muestra qué puede hacer una red neuronal clásica para aprender a corregir los errores de calibración de bits cuánticos. Se presenta también a <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a>, un marco de trabajo de Python para crear, editar e invocar circuitos de cuántica de escala intermedia ruidosa (NISQ, Noisy Intermediate Scale Quantum) y demostrar cómo Cirq se conecta con TensorFlow Quantum.

## Preparación

In [ ]:
!pip install tensorflow==2.7.0

Instalar TensorFlow Quantum:

In [ ]:
!pip install tensorflow-quantum==0.7.2

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

Ahora, hay que importar TensorFlow y las dependencias del módulo:

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 1. Lo básico

### 1.1 Cirq y los circuitos cuánticos parametrizados

Antes de explorar TensorFlow Quantum (TFQ), echemos un vistazo a algunos conceptos básicos sobre <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a>. Cirq es una biblioteca Python para el cálculo cuántico de Google. Se usa para definir circuitos, incluidas las puertas parametrizadas y estáticas.

Cirq usa símbolos <a target="_blank" href="https://www.sympy.org" class="external">SymPy</a> para representar los parámetros libres.

In [ ]:
a, b = sympy.symbols('a b')

El siguiente código crea un circuito de dos bits cuánticos usando esos parámetros:

In [ ]:
# Create two qubits
q0, q1 = cirq.GridQubit.rect(1, 2)

# Create a circuit on these qubits using the parameters you created above.
circuit = cirq.Circuit(
    cirq.rx(a).on(q0),
    cirq.ry(b).on(q1), cirq.CNOT(control=q0, target=q1))

SVGCircuit(circuit)

Para evaluar los circuitos, se puede usar la interfaz `cirq.Simulator`. Los parámetros libres de un circuito se reemplazan por números específicos, al pasar como entrada en un objeto `cirq.ParamResolver`. El siguiente código calcula el vector de estado sin procesar del circuito parametrizado:

In [ ]:
# Calculate a state vector with a=0.5 and b=-0.5.
resolver = cirq.ParamResolver({a: 0.5, b: -0.5})
output_state_vector = cirq.Simulator().simulate(circuit, resolver).final_state_vector
output_state_vector

No se puede acceder directamente a los vectores de estado fuera de la simulación (observemos los números complejos de la salida que figura arriba). Para ser físicamente realistas, deberá especificar una medición que convierta a un vector de estado en un número real que las computadoras clásicas puedan entender. Cirq especifica las mediciones mediante combinaciones de los <a target="_blank" href="https://en.wikipedia.org/wiki/Pauli_matrices" class="external">operadores Pauli</a> $\hat{X}$, $\hat{Y}$ y $\hat{Z}$. A modo ilustrativo, el siguiente código mide $\hat{Z}_0$ y $\frac{1}{2}\hat{Z}_0 + \hat{X}_1$ en el vector de estado que acabamos de simular:

In [ ]:
z0 = cirq.Z(q0)

qubit_map={q0: 0, q1: 1}

z0.expectation_from_state_vector(output_state_vector, qubit_map).real

In [ ]:
z0x1 = 0.5 * z0 + cirq.X(q1)

z0x1.expectation_from_state_vector(output_state_vector, qubit_map).real

### 1.2 Circuitos cuánticos como tensores

TensorFlow Quantum (TFQ) brinda `tfq.convert_to_tensor`, una función que convierte objetos Cirq en tensores. Gracias a ello, es posible enviar objetos Cirq a nuestras <a target="_blank" href="https://www.tensorflow.org/quantum/api_docs/python/tfq/layers">capas cuánticas</a> y <a target="_blank" href="https://www.tensorflow.org/quantum/api_docs/python/tfq/get_expectation_op">operaciones cuánticas</a>. La función se puede llamar en listas o arreglos de circuitos Cirq y Paulis Cirq:

In [ ]:
# Rank 1 tensor containing 1 circuit.
circuit_tensor = tfq.convert_to_tensor([circuit])

print(circuit_tensor.shape)
print(circuit_tensor.dtype)

De este modo, los objetos Cirq se codifican como tensores `tf.string` que las operaciones `tfq` decodifican según sea necesario.

In [ ]:
# Rank 1 tensor containing 2 Pauli operators.
pauli_tensor = tfq.convert_to_tensor([z0, z0x1])
pauli_tensor.shape

### 1.3 Simulación de circuito de loteo

TFQ proporciona métodos para calcular los valores de expectativa, las muestras y los vectores de estado. Por ahora, centrémonos en los *valores de expectativa*.

La interfaz de nivel más alto para calcular los valores de expectativa es la capa `tfq.layers.Expectation`, que es un `tf.keras.Layer`. En su forma más simple, esta capa es equivalente a simular un circuito parametrizado sobre muchos `cirq.ParamResolvers`; pero TFQ permite agrupar en lotes siguiendo la semántica de TensorFlow, y los circuitos se simulan con código C++ eficiente.

Creamos un lote de valores para sustituirlo para nuestros parámetros `a` y `b`:

In [ ]:
batch_vals = np.array(np.random.uniform(0, 2 * np.pi, (5, 2)), dtype=float)

Para agrupar la ejecución de circuitos en lotes sobre valores de parámetros en Cirq se requiere de un ciclo:

In [ ]:
cirq_results = []
cirq_simulator = cirq.Simulator()

for vals in batch_vals:
    resolver = cirq.ParamResolver({a: vals[0], b: vals[1]})
    final_state_vector = cirq_simulator.simulate(circuit, resolver).final_state_vector
    cirq_results.append(
        [z0.expectation_from_state_vector(final_state_vector, {
            q0: 0,
            q1: 1
        }).real])

print('cirq batch results: \n {}'.format(np.array(cirq_results)))

La misma operación se simplifica en TFQ:

In [ ]:
tfq.layers.Expectation()(circuit,
                         symbol_names=[a, b],
                         symbol_values=batch_vals,
                         operators=z0)

## 2. Optimización de cuántica clásica híbrida

Ahora que ya hemos visto los conceptos básicos, usemos TensorFlow Quantum para construir una *red neuronal de cuántica clásica híbrida*. Entrenaremos una red neuronal clásica para controlar un solo bit cuántico. El control se optimizará para preparar correctamente el bit cuántico en los estados `0` o `1`, superando un error de calibración semántica simulado. En esta figura se muestra la arquitectura:

<img src="./images/nn_control1.png" width="1000">

Aún sin una red neuronal, este es un problema muy sencillo de resolver. El tema es similar al de los problemas de control cuántico real que probablemente ya haya resuelto con TFQ. En este caso, se muestra un ejemplo completo de un cálculo clásico cuántico con la capa `tfq.layers.ControlledPQC` (de circuito cuántico parametrizado) dentro de un `tf.keras.Model`.

Para la implementación de este tutorial, la arquitectura se divide en 3 partes:

- El *circuito de entrada* o *circuito de puntos de datos*: las primeras tres puertas $R$.
- El *circuito controlado*: las otras tres puertas $R$.
- El *controlador*: la red neuronal clásica que determina los parámetros del circuito controlado.

### 2.1 La definición del circuito controlado

Definimos una rotación de un solo bit posible de aprender, como se indica en la figura anterior. Corresponderá a nuestro circuito controlado.

In [ ]:
# Parameters that the classical NN will feed values into.
control_params = sympy.symbols('theta_1 theta_2 theta_3')

# Create the parameterized circuit.
qubit = cirq.GridQubit(0, 0)
model_circuit = cirq.Circuit(
    cirq.rz(control_params[0])(qubit),
    cirq.ry(control_params[1])(qubit),
    cirq.rx(control_params[2])(qubit))

SVGCircuit(model_circuit)

### 2.2 El controlador

Ahora definimos la red del controlador: 

In [ ]:
# The classical neural network layers.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

Dado un lote de comandos, el controlador emite un lote de señales de control para el circuito controlado.

El controlador se inicializa aleatoriamente para que estas salidas todavía no sean útiles.

In [ ]:
controller(tf.constant([[0.0],[1.0]])).numpy()

### 2.3 Conexión del controlador al circuito

Use `tfq` para conectar el controlador al circuito controlado, como un solo `keras.Model`.

Consulte la [guía sobre la API funcional Keras](https://www.tensorflow.org/guide/keras/functional), para más información sobre este estilo de definición de modelo.

Primero, defina las entradas del modelo: 

In [ ]:
# This input is the simulated miscalibration that the model will learn to correct.
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.string,
                                name='circuits_input')

# Commands will be either `0` or `1`, specifying the state to set the qubit to.
commands_input = tf.keras.Input(shape=(1,),
                                dtype=tf.dtypes.float32,
                                name='commands_input')


A continuación, aplique las operaciones a estas entradas, para definir el cálculo.

In [ ]:
dense_2 = controller(commands_input)

# TFQ layer for classically controlled circuits.
expectation_layer = tfq.layers.ControlledPQC(model_circuit,
                                             # Observe Z
                                             operators = cirq.Z(qubit))
expectation = expectation_layer([circuits_input, dense_2])

Ahora, empaquete este cálculo como un `tf.keras.Model`:

In [ ]:
# The full Keras model is built from our layers.
model = tf.keras.Model(inputs=[circuits_input, commands_input],
                       outputs=expectation)

En el gráfico del modelo que se muestra a continuación se indica la arquitectura de la red. Compare este gráfico del modelo con el diagrama de la arquitectura para verificar que sean correctos.

Nota: Probablemente sea necesario instalar el paquete `graphviz` en el sistema.

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=70)

Este modelo toma dos entradas: los comandos para el controlador y las entradas del circuito cuyas salidas intenta corregir el controlador. 

### 2.4 El conjunto de datos

El modelo intenta emitir el valor de medición correcto de $\hat{Z}$ para cada comando. Los comandos y los valores correctos se definen a continuación.

In [ ]:
# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired Z expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

Este no es el conjunto completo de datos de entrenamiento para esta tarea. Cada punto de dato del conjunto de datos también necesita un circuito de entrada.

### 2.4 La definición del circuito de entrada

El siguiente circuito de entrada define a un error de calibración aleatorio que el modelo aprenderá a corregir.

In [ ]:
random_rotations = np.random.uniform(0, 2 * np.pi, 3)
noisy_preparation = cirq.Circuit(
  cirq.rx(random_rotations[0])(qubit),
  cirq.ry(random_rotations[1])(qubit),
  cirq.rz(random_rotations[2])(qubit)
)
datapoint_circuits = tfq.convert_to_tensor([
  noisy_preparation
] * 2)  # Make two copied of this circuit

Hay dos copias del circuito, una para cada punto de dato.

In [ ]:
datapoint_circuits.shape

### 2.5 El entrenamiento

Con las entradas definidas se puede probar el funcionamiento del modelo `tfq`.

In [ ]:
model([datapoint_circuits, commands]).numpy()

Ahora, ejecute un proceso de entrenamiento estándar para ajustar estos valores con respecto a las `expected_outputs`.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)
history = model.fit(x=[datapoint_circuits, commands],
                    y=expected_outputs,
                    epochs=30,
                    verbose=0)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

En este gráfico se puede ver que la red neuronal ha aprendido a superar el error sistemático de calibración.

### 2.6 Verificación de las salidas

En esta oportunidad, usamos el modelo entrenado para corregir errores de calibración de bits cuánticos. Con Cirq:

In [ ]:
def check_error(command_values, desired_values):
  """Based on the value in `command_value` see how well you could prepare
  the full circuit to have `desired_value` when taking expectation w.r.t. Z."""
  params_to_prepare_output = controller(command_values).numpy()
  full_circuit = noisy_preparation + model_circuit

  # Test how well you can prepare a state to get expectation the expectation
  # value in `desired_values`
  for index in [0, 1]:
    state = cirq_simulator.simulate(
        full_circuit,
        {s:v for (s,v) in zip(control_params, params_to_prepare_output[index])}
    ).final_state_vector
    expt = cirq.Z(qubit).expectation_from_state_vector(state, {qubit: 0}).real
    print(f'For a desired output (expectation) of {desired_values[index]} with'
          f' noisy preparation, the controller\nnetwork found the following '
          f'values for theta: {params_to_prepare_output[index]}\nWhich gives an'
          f' actual expectation of: {expt}\n')


check_error(commands, expected_outputs)

El valor de la función de pérdida durante el entrenamiento aporta una idea aproximada de en qué medida el modelo está aprendiendo bien. Mientras menor sea la pérdida, más se acercarán los valores de expectativa de la celda anterior a los `desired_values`. Si los valores del parámetro no son de su interés, siempre existe la posibilidad de controlar las salidas de arriba con `tfq`:

In [ ]:
model([datapoint_circuits, commands])

## 3 Aprendizaje de la preparación de autoestados de operadores diferentes

La elección de los autoestados (<em>eigenstates</em>) $\pm \hat{Z}$ correspondientes a 1 y 0 fue arbitraria. Se podría haber querido que 1 correspondiera al autoestado $+ \hat{Z}$ y que 0 correspondiera al autoestado $-\hat{X}$. Una manera de cumplir con esto es mediante la especificación de diferentes operadores de medición para cada comando, tal como se indica en la figura a continuación:

<img src="./images/nn_control2.png" width="1000">

Para lograrlo, es necesario usar <code>tfq.layers.Expectation</code>. Ahora, la entrada ha crecido e incluye tres objetos: circuito, comando y operador. La salida sigue siendo el valor de expectativa.

### 3.1 Definición del modelo nuevo

Echemos un vistazo al modelo para cumplir con esta tarea:

In [ ]:
# Define inputs.
commands_input = tf.keras.layers.Input(shape=(1),
                                       dtype=tf.dtypes.float32,
                                       name='commands_input')
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.dtypes.string,
                                name='circuits_input')
operators_input = tf.keras.Input(shape=(1,),
                                 dtype=tf.dtypes.string,
                                 name='operators_input')

Esta es la red del controlador:

In [ ]:
# Define classical NN.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

Combinemos el circuito y el controlador en un solo `keras.Model` con `tfq`:

In [ ]:
dense_2 = controller(commands_input)

# Since you aren't using a PQC or ControlledPQC you must append
# your model circuit onto the datapoint circuit tensor manually.
full_circuit = tfq.layers.AddCircuit()(circuits_input, append=model_circuit)
expectation_output = tfq.layers.Expectation()(full_circuit,
                                              symbol_names=control_params,
                                              symbol_values=dense_2,
                                              operators=operators_input)

# Contruct your Keras model.
two_axis_control_model = tf.keras.Model(
    inputs=[circuits_input, commands_input, operators_input],
    outputs=[expectation_output])

### 3.2 El conjunto de datos

Ahora, también incluya los operadores que desee medir para cada punto de dato que proporcione al `model_circuit`:

In [ ]:
# The operators to measure, for each command.
operator_data = tfq.convert_to_tensor([[cirq.X(qubit)], [cirq.Z(qubit)]])

# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

### 3.3 El entrenamiento

Ahora que tiene entradas y salidas nuevas se puede volver a entrenar con keras.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()

two_axis_control_model.compile(optimizer=optimizer, loss=loss)

history = two_axis_control_model.fit(
    x=[datapoint_circuits, commands, operator_data],
    y=expected_outputs,
    epochs=30,
    verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

La función de pérdida ha caído a cero.

El `controller` está disponible como un modelo independiente. Llame al controlador y verifique la respuesta que da a cada señal de comando. Probablemente no sea sencillo comparar correctamente estas salidas con el contenido de las `random_rotations`.

In [ ]:
controller.predict(np.array([0,1]))

Vea si puede adaptar la función `check_error` de su primer modelo para trabajar con esta nueva arquitectura de modelo.